## EDA - Exploratory Data Analysis
Dataset 0 is the Univariated dataset from our data.  
The only variable in the dataset 0 is 'maximum temperature' in station A612 - Vitoria, capital of Espirito Santo.  

<a href="https://ibb.co/ZccxLYV"><img src="https://i.ibb.co/KVVW0LX/A612-Station.png" alt="A612-Station" border="0"></a>

### Importing Libs

In [ ]:
## basic libs
import pandas as pd
import numpy as np

## ploting libs
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from statsmodels.tsa.stattools import acf,pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm

from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.arima.api import ARIMA
from jh_utils.timeseries import time_series_dataframe

figsize = (20,8)

### Reading data
Dataset 0 has 46 columns, for the Exploratory Data Analysis we just need the Index and the Maximum Temperature from Station A612.  
All other columns are dummies just needed to make the autoregressive models. To get the month and year information we will use methods from Pandas

In [ ]:
df = pd.read_csv('../input/brazil-weather-climate-data/ES_0.csv')
df = df.iloc[:,[0,7]]
df.date_time = pd.to_datetime(df.date_time)
df['month'] = df.date_time.dt.month_name()
df['year'] = df.date_time.dt.year
df['hour'] = df.date_time.dt.hour

In [ ]:
_ = df[df["date_time"] >= '2015-01-01'].resample('1d', on='date_time').mean().iloc[:,0].plot(figsize= figsize).set_title('Mean temperature aggregated by mean 3 days',fontsize=20)

In [ ]:
plt.figure(figsize=figsize)
_ = sns.histplot(df['A612 - temperatura maxima na hora ant. (aut) (°c)']).set_title('Maximum Temperature Histogram',fontsize=20)

In [ ]:
plt.figure(figsize=figsize)
_ = sns.boxplot(data=df,y = 'A612 - temperatura maxima na hora ant. (aut) (°c)', x='month').set_title('Month`s Boxplots',fontsize=20)

In [ ]:
plt.figure(figsize=figsize)
_ = sns.boxplot(data=df,y = 'A612 - temperatura maxima na hora ant. (aut) (°c)', x='year').set_title('Years`s Boxplots',fontsize=20)

In [ ]:
plt.figure(figsize=figsize)
_ = sns.boxplot(data=df,y = 'A612 - temperatura maxima na hora ant. (aut) (°c)', x='hour').set_title('Hour`s Boxplots',fontsize=20)

## Acf and Pacf from Linear Regression errors

In [ ]:
df = pd.read_csv('../input/brazil-weather-climate-data/ES_0.csv')
## remove index
df = df.iloc[:,7]

In [ ]:
columns_without_weekday = list(map(lambda x: x!=True,df.columns.str.contains('weekday').tolist()))

In [ ]:
y = df.iloc[:,0]
x = df.iloc[:,columns_without_weekday].iloc[:,1:-4]
model = sm.OLS(y,x)
model_fit = model.fit()
model_fit.summary()

In [ ]:
_ = plot_acf(model_fit.resid)

In [ ]:
_ = plot_pacf(model_fit.resid)

### Autoregressive Model
Now that we know that we have a sequence information problem we will use an Autoregressive Model

In [ ]:
arima = ARIMA(y,x,order=(3,0,1))

In [ ]:
arima_fit = arima.fit()

In [ ]:
arima_fit.summary